In [1]:
# install duckdb package
import duckdb

In [5]:
# show all available schemas and tables
conn = duckdb.connect("./data/lake/database.duckdb")
conn.sql("""
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
    ORDER BY table_schema, table_name;
""").pl()

table_schema,table_name
str,str
"""public""","""climate_atmosphere_raw"""
"""public""","""disease_pidsr_totals_raw"""
"""public""","""location_raw"""
"""public""","""project_cchain_climate_atmosph…"
"""public""","""project_cchain_disease_pidsr_t…"
"""public""","""waqi_airquality_raw"""


In [6]:
# sample query
conn = duckdb.connect("./data/lake/database.duckdb")
waqi_airquality_raw = conn.sql("""
    SELECT *
    FROM public.waqi_airquality_raw
    ORDER BY date
    LIMIT 50;
    """).pl()
waqi_airquality_raw

date,pm25,pm10,o3,no2,so2,co
str,str,str,str,str,str,str
"""01-Apr-14""",null,"""89""","""12""","""33""","""28""","""15"""
"""01-Apr-15""",null,"""45""","""48""","""14""","""14""","""9"""
"""01-Apr-16""","""134""","""72""","""54""","""27""","""1""","""14"""
"""01-Apr-17""","""145""","""35""","""17""","""5""",""" """,""" """
"""01-Apr-18""","""135""","""64""","""30""","""6""","""11""",""" """
…,…,…,…,…,…,…
"""01-Jan-14""",null,"""56""","""43""","""20""","""3""","""9"""
"""01-Jan-15""",null,"""39""","""37""","""15""","""9""","""10"""
"""01-Jan-17""",null,"""22""","""12""","""4""",""" """,""" """
